In [1]:
from deeprobust.graph.data import Dataset
from scapin import Scapin
import numpy as np
import torch
import argparse
import utils

seed = 0
device = "cuda:0"
th = 0.07
ptb_rate = 0.03
model = "LPA"

np.random.seed(seed)
torch.manual_seed(seed)
if device != 'cpu':
    torch.cuda.manual_seed(seed)

In [2]:
adj, features, labels = utils.load_npz_to_sparse_graph(f"ms_dataset/ms_academic_cs.npz")
features = np.asarray(features.todense())
num_nodes = len(labels)

train_idx, valid_idx, test_idx = utils.get_train_val_test(num_nodes, val_size=0.1, test_size=0.8, stratify=labels,
                                                          seed=seed)

split_idx = {
    "train": train_idx,
    "valid": valid_idx,
    "test": test_idx
}

In [3]:
res = Scapin.evaluate(adj=adj, features=features, labels=labels, split_idx=split_idx, model=model)

print(f"original acc: {res}")

original acc: 0.8601622581481934


In [4]:
perturbed_adj = Scapin.generate(adj=adj, split_idx=split_idx, ptb_rate=ptb_rate, th=th)
perturbed_adj

<18333x18333 sparse matrix of type '<class 'numpy.float32'>'
	with 168700 stored elements in List of Lists format>

In [5]:
perturbed_res = Scapin.evaluate(adj=perturbed_adj, features=features, labels=labels, split_idx=split_idx, model=model)

print(f"perturbed acc: {perturbed_res}")

perturbed acc: 0.7765732407569885
